# PIERS Data Cleaning

This notebook addresses missing data and other issues in the PIERS BOL data. This work is predominantly exploratory in nature, and some of these processes may eventually move to the ETL step. 

In [1]:
import pandas as pd
import polars as pl
import plotly_express as px
import missingno as msno

#display settings
pd.set_option('display.max_columns', None)

#enable string cache for polars categoricals
pl.enable_string_cache()

Convenience steps:
- create 2-digit hs codes (note this naievely takes the first two digits from the column, which contains multiple hs codes)
- create year and month (e.g., 202305) columns 
- replace zero values with nulls in volume columns
- create voyage_id from voyage number and vessel IMO code. 
- add direction column showing 'import' or 'export' to allow easy merging

In [2]:
#set paths
imports_path = 'data/clean/imports/'
exports_path = 'data/clean/exports/'
#init lazy dataframes
imports_lf = (
    pl.scan_parquet(imports_path+'*.parquet', parallel='columns')
    .with_columns([
        #create 2-digit hs code
        pl.col('hs_code').str.slice(0, length=2).alias('hs_2d'),
        #create year and month columns
        pl.col('date_arrival').dt.year().alias('year'),
        pl.col('date_arrival').dt.strftime('%Y%m').alias('month'),
        #convert zero volume values to null
        pl.col('teus').replace(0,None),
        pl.col('weight').replace(0,None),
        pl.col('qty').replace(0,None),
        #create voyage_id from vessel IMO and voyage number
        (pl.col('vessel_id').cast(pl.Utf8)+'_'+pl.col('voyage_number').cast(pl.Utf8)).cast(pl.Categorical).alias('voyage_id'),
        #create direction column
        pl.lit('import').cast(pl.Categorical).alias('direction')
        ])
    )
exports_lf = (
    pl.scan_parquet(exports_path+'piers_exports_complete.parquet', parallel='columns')
    .with_columns([
        #create 2-digit hs code
        pl.col('hs_code').str.slice(0, length=2).alias('hs_2d'),
        #create year and month columns
        pl.col('date_departure').dt.year().alias('year'),
        pl.col('date_departure').dt.strftime('%Y%m').alias('month'),
        #convert zero volume values to null
        pl.col('teus').replace(0,None),
        pl.col('weight').replace(0,None),
        pl.col('qty').replace(0,None),
        #create voyage_id from vessel IMO and voyage number
        (pl.col('vessel_id').cast(pl.Utf8)+'_'+pl.col('voyage_number').cast(pl.Utf8)).cast(pl.Categorical).alias('voyage_id'),
        #create direction column
        pl.lit('export').cast(pl.Categorical).alias('direction')
        ])
    )

## Notebook functions

In [3]:
#fill nulls in volume cols with mean
def fill_volume(lf):
    '''ad hod function to fill volume columns with their means'''
    return (
        lf
        .with_columns([
            pl.col('teus').replace(0,None).fill_null(strategy='mean'),
            pl.col('weight').replace(0,None).fill_null(strategy='mean'),
            pl.col('qty').replace(0,None).fill_null(strategy='mean')
            ])
        )

#plotly graph inspecting nulls over time by group
def nulls_over_time_plotly(data_lf, group_var, time_var, value_var, title=False):
    '''
    Plots proportion of null values over time by group.
    INPUTS:
        data_lf - polars lazyframe containing the relevant data
        group_var - str - the name of the column by which to group
        time_var - str - the name of the time column (e.g., year, month) over which values will be counted
        value_var - str - the name of the column containing the variable in question
        title (default=False) - str - the title of the graph
    OUTPUT:
        a plotly express figure
    DEPENDS ON:
        polars
        plotly express 
    '''
    df = (
        #select relevant columns
        data_lf.select([group_var, time_var, value_var])
        #group by, creating null count and non-null count cols
        .group_by(group_var, time_var)
        .agg([pl.col(value_var).null_count().alias('null_count'),
                pl.col(value_var).count().alias('count')])
        #compute percent null and fill new column
        .with_columns((pl.col('null_count')/(pl.col('count')+pl.col('null_count'))).alias('null_percent'))
        #cast group col to string to allow sensible ordering of legend
        .cast({group_var:pl.Utf8})
        #sort by date (to allow proper visualization of lines) and group (for legend ordering) 
        .sort(time_var, group_var)
    ).collect()
    #plot
    fig = px.line(
        data_frame=df,
        x=time_var, y='null_percent',
        color=group_var,
        title= 'Count of nulls over time by source frame.' if not title else title
    )
    fig.show()

#plotly graph inspecting nulls by group
def nulls_by_group_plotly(data_lf, group_var, value_var, title=False):
    '''Plots proportion of null values in the given groups'''
    df = (
        data_lf.select([group_var, value_var])
        .group_by(group_var)
        .agg([pl.col(value_var).null_count().alias('null_count'),
                pl.col(value_var).count().alias('count')])
        .with_columns((pl.col('null_count')/(pl.col('count')+pl.col('null_count'))).alias('null_percent'))
        .cast({group_var:pl.Utf8})
        .sort('null_percent', descending=True)
    ).collect()
    #plot
    fig = px.bar(
        data_frame=df,
        x=group_var, y='null_percent',
        title= 'Null percent by group.' if not title else title
    )
    fig.show()

#fill nulls over groups given a single unique value per group
def fill_nulls_by_group(data_lf, group_vars, val_var):
    '''Fills null values by group if and only if the val_var for that group contains exactly one non-null unique value.
    INPUTS:
        data_lf - polars lazyframe containing the relevant data
        group_vars - iterable - the names of the columns by which groups will be created
        val_var - string - the name of the column in which nulls will be filled
    OUTPUT:
        filled_lf - the resultant lazyframe 
    DEPENDS ON:
        polars - current version written in polars 0.20.1
    '''
    filled_lf = (
        data_lf.with_columns(
            #if the group contains exactly one unique value: 
            pl.when(pl.col(val_var).drop_nulls().unique(maintain_order=True).len().over(group_vars)==1)
            #then fill the group with that value
            .then(pl.col(val_var).fill_null(pl.col(val_var).drop_nulls().unique(maintain_order=True).first().over(group_vars)))
            #otherwise do nothing
            .otherwise(pl.col(val_var))
            )
        )
    return filled_lf

## Missing Data


In [ ]:
print('Missing Values Matrix for Exports. \nThe vertical axis corresponds to date, with earliest at the top.  \nNote the daily aggregation counts the row as non-null if any value that day is non-null.')
msno.matrix(
    exports_lf
    #convert all columns except date to boolean False=null
    .with_columns(pl.all().exclude('date_departure').is_not_null())
    #aggregate by day to stay within visualization limitations 
    .group_by('date_departure').sum()
    .sort('date_departure')
    #convert False values back to null 
    .with_columns(pl.all().replace(0,None))
    .collect()
    .to_pandas()
    );

In [ ]:
print('Missing Values Matrix for Imports. \nThe vertical axis corresponds to date, with earliest at the top. \nNote the daily aggregation counts the row as non-null if any value that day is non-null.')
msno.matrix(
    imports_lf
    #convert all columns except date to boolean False=null
    .with_columns(pl.all().exclude('date_arrival').is_not_null())
    #aggregate by day to stay within visualization limitations 
    .group_by('date_arrival').sum()
    .sort('date_arrival')
    #convert False values back to null 
    .with_columns(pl.all().replace(0,None))
    .collect()
    .to_pandas()
    );

### Zero values in TEU and other volume columns

For the time being, we fill the zero (i.e., missing) volume data with the mean from non-zero observations. This step is expected to change as beter metadata is gathered. 

In [4]:
imports_lf = fill_volume(imports_lf)
exports_lf = fill_volume(exports_lf)    

### Missing Vessel and Voyage IDs

Identifying unique vessels and voyages is critical to studying how long each vessel remains in port, how often carriers utilize alliances (e.g., what proportion of a vessel's cargo is from another carrier), and other quality metrics. The issue is especially concerning given the high proportion (~30% in some time periods) of missing data, as can be seen below.

In [ ]:
lf = pl.concat(
    [imports_lf.select('direction', 'month', 'vessel_id'),
    exports_lf.select('direction', 'month', 'vessel_id')]
)

nulls_over_time_plotly(
    data_lf=lf,
    group_var='direction',
    time_var='month',
    value_var='vessel_id',
    title='Proportion of Missing Vessel IDs over time (original data).')

In [ ]:
lf = pl.concat(
    [imports_lf.select('direction', 'month', 'vessel_name'),
    exports_lf.select('direction', 'month', 'vessel_name')]
)

nulls_over_time_plotly(
    data_lf=lf,
    group_var='direction',
    time_var='month',
    value_var='vessel_name',
    title='Proportion of Missing Vessel Names over time (original data).')

In [ ]:
lf = pl.concat(
    [imports_lf.select('direction', 'month', 'voyage_id'),
    exports_lf.select('direction', 'month', 'voyage_id')]
)

nulls_over_time_plotly(
    data_lf=lf,
    group_var='direction',
    time_var='month',
    value_var='voyage_id',
    title='Proportion of Missing Voyage IDs over time (original data).')

## Primary Carrier

For any given voyage, we assign primary carrier based on the proportion of TEUs from each carrier imported/exported on that voyage (max proportion is primary carrier). 

In [ ]:
primary_scac_lf = (
    imports_lf
    #select relevant cols
    .select('voyage_id', 'carrier_scac', 'teus')
    #ignore bols with missing data
    .drop_nulls()
    #sum up TEUs by voyage and carrier
    .group_by('voyage_id', 'carrier_scac')
    .agg(pl.col('teus').sum().alias('sum_teus'))
    #choose the carrier with the most TEUs on each voyage
    .sort('sum_teus', descending=True)
    .group_by('voyage_id')
    .agg(pl.col('carrier_scac').first().alias('primary_scac'))
    )

imports_lf = imports_lf.join(primary_scac_lf, how='left', on='voyage_id')

In [5]:
#find primary carrier per voyage and merge back into main lf
def add_primary_carrier(lf):
    lf2 = (
        lf
        #select relevant cols
        .select('voyage_id', 'carrier_scac', 'teus')
        #ignore bols with missing data
        .drop_nulls()
        #sum up TEUs by voyage and carrier
        .group_by('voyage_id', 'carrier_scac')
        .agg(pl.col('teus').sum().alias('sum_teus'))
        #choose the carrier with the most TEUs on each voyage
        .sort('sum_teus', descending=True)
        .group_by('voyage_id')
        .agg(pl.col('carrier_scac').first().alias('primary_scac'))
    )
    lf = (
        #add primary scac column to main lf
        lf.join(lf2, how='left', on='voyage_id')
        #add bool col if bol is from primary carrier
        .with_columns((pl.col('carrier_scac')!=pl.col('primary_scac')).alias('shared_cargo'))
    )

    return lf

imports_lf = add_primary_carrier(imports_lf)
exports_lf = add_primary_carrier(exports_lf)

In [10]:
def sharing_over_time_plotly(data_lf, group_var, title=False):
    '''
    Plots proportion of shared cargo over time (months) by group_var.
    INPUTS:
        data_lf - polars lazyframe containing the relevant data
        group_var - str - the name of the column by which to group
        title (default=False) - str - the title of the graph
    OUTPUT:
        a plotly express figure
    DEPENDS ON:
        polars
        plotly express 
    '''
    df = (
        #select relevant columns
        data_lf.select([group_var, 'month', 'shared_cargo', 'teus'])
        #sum teus over each group-month-shared 
        .group_by(group_var, 'month')
        .agg(
            (pl.col('teus')*pl.col('shared_cargo')).sum().alias('total_shared'),
            pl.col('teus').sum().alias('total_teus')
        )
        #create proportion shared
        .with_columns((pl.col('total_shared')/pl.col('total_teus')).alias('prop_shared'))
        #cast group col to string to allow sensible ordering of legend
        .cast({group_var:pl.Utf8})
        #sort by date (to allow proper visualization of lines) and group (for legend ordering) 
        .sort('month', group_var)
    ).collect()

    #plot
    fig = px.line(
        data_frame=df,
        x='month', y='prop_shared',
        color=group_var,
        title= 'Count of nulls over time by source frame.' if not title else title
    )
    fig.show()

In [12]:
lf = pl.concat([imports_lf,exports_lf], how='diagonal')

sharing_over_time_plotly(
    data_lf=lf,
    group_var='direction',
    title='Proportion of TEUs on shared vessels over time.')

In [13]:
lf = pl.concat(
    [imports_lf.select('direction', 'month', 'voyage_id'),
    exports_lf.select('direction', 'month', 'voyage_id')]
)

nulls_over_time_plotly(
    data_lf=lf,
    group_var='direction',
    time_var='month',
    value_var='voyage_id',
    title='Proportion of Missing Voyage IDs over time (original data).')

In [17]:
top_ports = (
    exports_lf.group_by('departure_port_name')
    .agg(pl.col('teus').sum())
    .sort('teus', descending=True)
    .select('departure_port_name')
    .limit(10)
    .collect()
    .to_series()
)

lf = exports_lf.filter(pl.col('departure_port_name').is_in(top_ports))

sharing_over_time_plotly(
    data_lf=lf,
    group_var='departure_port_name',
    title='Proportion of TEUs on shared vessels over time (Top 10 export ports by total TEUs).')

## Route IDs

Create route ID column from arrival and departure port codes. 

In [ ]:
#create route_id column
imports_lf = (
    imports_lf.with_columns(
        (pl.col('departure_port_code').cast(pl.Utf8)+'_'+pl.col('arrival_port_code').cast(pl.Utf8))
        .cast(pl.Categorical)
        .alias('route_id')
        ))
exports_lf = (
    exports_lf.with_columns(
        (pl.col('departure_port_code').cast(pl.Utf8)+'_'+pl.col('arrival_port_code').cast(pl.Utf8))
        .cast(pl.Categorical)
        .alias('route_id')
        ))

## Days in port

For each arrival, we want to know how many days the ship stays in port before departing. 


In [ ]:
ex_lf = (
    exports_lf.select('voyage_id', 'vessel_id', 'date_departure', 'departure_port_code', 'route_id')
    .rename({'voyage_id':'export_voyage_id'})
    .with_columns((pl.col('date_departure').cast(pl.Utf8)+'_'+pl.col('vessel_id').cast(pl.Utf8)+'_'+pl.col('departure_port_code').cast(pl.Utf8)).alias('departure_id'))
    )
im_lf = (
    imports_lf.select('voyage_id', 'vessel_id', 'date_arrival', 'arrival_port_code', 'route_id')
    .rename({'voyage_id':'import_voyage_id'})
    .with_columns((pl.col('date_arrival').cast(pl.Utf8)+'_'+pl.col('vessel_id').cast(pl.Utf8)+'_'+pl.col('arrival_port_code').cast(pl.Utf8)).alias('arrival_id'))
    )

lf = ex_lf.join(im_lf, on='voyage_id')

## Export Clean data to parquet


In [ ]:
#piers_lf.sink_parquet('data/piers_clean.parquet') -- NOTE the above queries are apparently not supported for sinking at this time. 